# Seminario de programacion
# Simulación del Juego llamado "Guayabita"

### Integrantes



Juan Speck



Julian Buitrago

##### El juego llamada guayabita consiste en cada jugador lanza un dado. luego de obtener el resultado, apuesta el valor del pote o el valor de la apuesta inicial, según la probabilidad de que pueda ganar. si el primer resultado es 1 el jugador automáticamente pierde la apuesta, y si es 6 el jugador pasa de turno sin perder saldo. El experimento aleatorio consiste en ganar la apuesta o el pote, y cada ronda se ejecutará hasta que 1 jugador gane o en su defecto todos lo jugadores no tengan el saldo suficiente para seguir apostando, por lo que perderían automáticamente.

##### Para el desarrollo de este proyecto, se usó como base la guía "Guayabita", documento con los requerimientos para el desarrollo de este programa, y adicional el video que tiene como título 'Explicación Como jugar "Guayabita", (dados)'.

<https://www.youtube.com/watch?v=nj2bZWBQROg&ab_channel=JhonnyRom%C3%A1nLunaFuentes>

### Librerias

In [6]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

#pd.set_option('display.max_rows', 1000)

### Supuestos

1. Dentro de las reglas definidas en este programa, el jugador siempre tendrá que apostar, pero según la probabilidad de ganar y tener el dinero necesario para cubrir la apuesta ira ya sea por el pote o por el valor inicial de apuesta.

2. Los jugadores perderán cuando ya no tengan dinero suficiente para suplir el valor de la apuesta o quede con saldo 0

3. Solo se simulará 11 horas de juego

4. La llegada de jugadores se manejará a través de una distribución de Poisson

5. Durante una jornada de "Guayabita" habrá mesas de 4 o 6 jugadores, las cuales serán seleccionadas de manera aleatoria cumpliendo una distribución normal

6. Durante una jornada de "Guayabita" habrá partidas que van a manejar la decisión de la apuesta de manera controlada con una probabilidad de 0.5 o con una distribución normal de media de 0.6 y desviación 0.1

7. En caso de la partida no tenga ningún ganador, la ganancia total se la lleva el casino (El pote de la ronda pasa a la ganancia del casino)

## Desarrollo

### Declaración de variables

In [7]:
#Espacio para creacion de variables
jugadores_partida=[4, 6] #Lista para determinar la cantidad de jugadores que puede tener una partida
cant_jugadores=0 #Variable para setiar la cantidad de jugadores que tiene cada partida
montos_apuesta=[5000,10000,20000,50000] #Se crea una lista con los posibles valores que se pueden apostar por cada partida
dado = [1,2,3,4,5,6] #Se crea una lista para definir los valores del dado
dado_probabilidad=[1/6, 1/6, 1/6, 1/6, 1/6, 1/6] #Se crea una lista para determinar la probabilidad de ocurrencia de cada uno de los valores del dada
num_dados=1 #Cantidad de dados que van a intervenir en la partida
pote=0 #El valor acumulado de las apuestas durante la partida
ganancia_casino=0 #Variable de acumulacion de lo que va recibiendo el casino
porc_ganan_casino=0.05 #Variable que define el porcentaje de ganancia del casino por partida
#iteracion_jugador=0 #Variable que contrala los jugadores del casino en 1 dia
partida=1
cola_jugadores=0 #Inicializacion de variable para establecer una cola de entrada de jugadores a cada partida

### Simulación de llegada de los jugadores

In [8]:
llegadas_hora=np.random.poisson(50,11) # Se utiliza un lambda de 50, que equivale a la tasa promedio de visitantes por hora, y el tamaño de 11 que equivale a el numero de sucesos en 11 horas
horas=0 #Numero de horas de la partida
j=0 #La iteracion de la partida de los jugadores

Lista_llegadas=[] #Declaración de la Lista donde se van a guardar los registros de la hora, minutos y minutos acumulados de la llegada de los jugadores desde la apertura del casino que 
for i in llegadas_hora: #Ciclo para recorrer el orden de llegada de los jugadores por hora
    horas+=1 #Hora desde apertura del casino
    Min=np.random.choice(np.arange(0,60),i) #Distribucion normal para asignar los minutos de llegada de cada jugador
    Min.sort() #Ordenar de manera ascendente los minutos
    Lista_llegadas.extend([(horas,j,(horas-1)*60+j) for j in Min]) #Alimentacion de la lista con la hora, minutos y minutos acumulados desde la apertura hasta el cierre del casino

DF=pd.DataFrame(Lista_llegadas)
DF.columns=['Hora','Minuto','Minutos desde apertura']

### Simulación de las partidas

In [9]:
#Ciclo indefinido para determinar la cantidad de partidas en una jornada, basado en la cantidad de jugadores que llegan en 11 horas
while cola_jugadores<len(DF):
    
    #print(f"Partida No {partida}")
    
    saldo_jugador=[] #Lista que almacena cuanto tiene acumulado los jugadores en cada partida
    estado_jugador=[] #Lista que determina si el jugador esta activo o ha perdido dentro de la partida
    ejecucion=0 #Variable que determina la cantidad de rondas que se van ejecutando por partida 
    valor_apuesta=np.random.choice(montos_apuesta) #Selecciona el valor de la apuesta de la partida y sigue una distribución normal
    pote=0 #Variable que contiene el valor del pote en cada partida

    cant_jugadores=np.random.choice(jugadores_partida) #Se define a través de una distr. normal la cantidad de jugadores por partida
    
    #Condicional para reasignar la cantidad de jugadores de la partida, en caso de que sena 6 y no hayan más jugadores en la lista, se setea en 4
    if cant_jugadores==6 and cola_jugadores+cant_jugadores>len(DF):
        cant_jugadores=4
    
    #Condicional para empezar las rondas de cada partida, siempre y cuando se cuente con los jugadores disponibles
    if cola_jugadores+cant_jugadores<=len(DF):

        #Ciclo definido para asignación de estados,montos y apuestas iniciales a cada jugador en las listas creadas y en el dataframe
        for i in range(cant_jugadores):
            saldo_jugador.append(100000)
            estado_jugador.append('Activo') #Todos los jugadores al inicio de la partida estan activos
            saldo_jugador[i]=saldo_jugador[i]-valor_apuesta
            pote=pote+valor_apuesta
            #Creacion de nuevas columnas en el dataframe con información relevante de la partida
            DF.loc[cola_jugadores+i, 'Partida'] = partida
            DF.loc[cola_jugadores+i, 'Inicio partida'] = DF.iloc[cola_jugadores:cola_jugadores+cant_jugadores]['Minutos desde apertura'].max()
            DF.loc[cola_jugadores+i, 'Cantidad jugadores'] = cant_jugadores
            DF.loc[cola_jugadores+i, 'Saldo jugador'] = saldo_jugador[i]
            DF.loc[cola_jugadores+i, 'Valor apuesta'] = valor_apuesta
            DF.loc[cola_jugadores+i, 'No de rondas'] = 0
            DF.loc[cola_jugadores+i, 'Estado jugador'] = 'Perdio'
          
        
        prob_apuesta=np.random.binomial(1, 0.5, size=None) #Variable que controlará la probabilidad de apuesta de los jugadores(La probabilidad de su apuesta depende de que el 
        #valor supere 0.5 o La probabilidad de su apuesta depende de un valor aleatorio con districbución normalcon media 6 y desviacion 0.1)
        
        while (pote>0) and (estado_jugador.count('Perdio')<cant_jugadores):
            
            #print(f"Ejecucion {ejecucion},", f"Perdidos: {estado_jugador.count('Perdio')}", cant_jugadores)
            
            for i in range(cant_jugadores):

                if saldo_jugador[i]>=valor_apuesta and pote>0:

                    #print(f"Jugador {i+1}")
                    
                    resultado_primer_dado = np.random.choice(dado, size=num_dados, p=dado_probabilidad)
                    
                    if prob_apuesta==0:
                        decision_apuesta_prob=(6-resultado_primer_dado)/6
                    else:
                        decision_apuesta_prob=np.random.normal(0.6, 0.1, size=None)

                    #Condicional para determinar basado en la distribucion con la que se calcula la decision de apuesta si el jugador va por el monto del "pote" o vuelve a 
                    #apostar el valor de la apuesta
                    if decision_apuesta_prob>=0.5 and saldo_jugador[i]>=pote:
                        decision_apuesta='Pote'
                    elif ((decision_apuesta_prob<0.5 and saldo_jugador[i]>=valor_apuesta) or (decision_apuesta_prob>=0.5 and saldo_jugador[i]>=valor_apuesta)):
                        decision_apuesta='Apuesta'

                    #print("Saldo=",saldo_jugador[i],"ganancia_casino=",ganancia_casino, "pote=",pote, "valor_apuesta=",valor_apuesta, "decision_apuesta_prob=", decision_apuesta_prob)
                    
                    resultado_segundo_dado  = np.random.choice(dado, size=num_dados, p=dado_probabilidad)
                    #Bloque de condicionales para determinar si el jugador gana o pierde en cada ronda
                    if resultado_primer_dado==1: #Condicional establecido en la documentacion, donde si saca 1 pierde y lleva el pote el valor de la apuesta
                        saldo_jugador[i]=saldo_jugador[i]-valor_apuesta
                        pote=pote+valor_apuesta
                    elif resultado_primer_dado==6:  #Condicional establecido en la documentacion, donde si saca 6 pasa el turno y no se afecta el saldo
                        saldo_jugador[i]=saldo_jugador[i]
                        #pote=pote
                    elif ((resultado_primer_dado>resultado_segundo_dado and decision_apuesta=='Pote') or (resultado_primer_dado==resultado_segundo_dado and decision_apuesta=='Pote')):
                        #Bloque que afecta el saldo en caso de perder y haya apostado el ir por el pote
                        saldo_jugador[i]=saldo_jugador[i]-pote
                        pote=pote+pote
                    elif resultado_primer_dado>resultado_segundo_dado and decision_apuesta=='Apuesta':
                        #Bloque que afecta el saldo en caso de perder y haya apostado el ir por la apuesta
                        saldo_jugador[i]=saldo_jugador[i]-valor_apuesta
                        pote=pote+valor_apuesta
                    elif resultado_primer_dado<resultado_segundo_dado and decision_apuesta=='Pote':
                        #Bloque que afecta el saldo en caso de ganar y haya apostado el ir por el pote, lo que garantiza la finalizacion de la partida an dejar el pote en $0
                        saldo_jugador[i]=saldo_jugador[i]+(pote*(1-porc_ganan_casino))
                        ganancia_casino=ganancia_casino+(pote*porc_ganan_casino)
                        estado_jugador[i]='Gano'
                        pote=0
                    elif  resultado_primer_dado<resultado_segundo_dado and decision_apuesta=='Apuesta':
                        #Bloque que afecta el saldo en caso de ganar y haya apostado el ir por la apuesta
                        saldo_jugador[i]=saldo_jugador[i]+(valor_apuesta*(1-porc_ganan_casino))
                        ganancia_casino=ganancia_casino+(valor_apuesta*porc_ganan_casino)
                        pote=pote-valor_apuesta
                    
                    #Condicional que controla cuando el jugador pierde, debido al saldo que le queda es inferior al valor de la apuesta inicial
                    if saldo_jugador[i]<valor_apuesta:
                        estado_jugador[i]='Perdio'

                    #print("Saldo=",saldo_jugador[i],"decision_apuesta=",decision_apuesta,  "resultado_primer_dado=",resultado_primer_dado, "resultado_segundo_dado=",resultado_segundo_dado,"ganancia_casino=",ganancia_casino, "pote=",pote, "Estado=", estado_jugador[i])
                    
                    #Asignacion del saldo del jugador en cada iteracion dentro del dataframe
                    DF.loc[cola_jugadores+i, 'Saldo jugador'] = saldo_jugador[i]
                    
                    #Asignacion del estado del jugador de la partida dentro del dataframe
                    if estado_jugador[i]!='Activo':
                        DF.loc[cola_jugadores+i, 'Estado jugador'] = estado_jugador[i]
                        
                    #Se almacena cuantas rondas alcanzo a jugar el jugador dentro de cada partida    
                    DF.loc[cola_jugadores+i, 'No de rondas'] = ejecucion+1
                    
                decision_apuesta='' #Se inicializa la decisión de la apuesta, para las siguientes rondas
            ejecucion=ejecucion+1   #Contabiliza la cantidad de rondas que hay por partida
        ganancia_casino=ganancia_casino+pote #Acumula la ganancia del casino con el valor del pote, si el pote es 0, la ganancia del casino se sigue manteniendo
        
        #Ciclo para almacenar en el dataframe la ganancia del casino por cada partida
        for i in range(cant_jugadores):
            DF.loc[cola_jugadores+i, 'Ganancia casino'] = ganancia_casino
            
        partida=partida+1 #Contabiliza la cantidad de partidas que hay
        cola_jugadores=cola_jugadores+cant_jugadores #Contabiliza la cola dfe espera de los jugadores para iniciar las nuevas partidas
        
    else:
        break
print(f"La ganancia del casino durante una jornada de guayabita de {round(DF['Minutos desde apertura'].max()/60)} horas es de=$",ganancia_casino)

La ganancia del casino durante una jornada de guayabita de 11 horas es de=$ 26757250.0


### Resultados de cada partida

In [10]:
DF

,Hora,Minuto,Minutos desde apertura,Partida,Inicio partida,Cantidad jugadores,Saldo jugador,Valor apuesta,No de rondas,Estado jugador,Ganancia casino
0,1,0,0,1.0,5.0,6.0,14000.0,20000.0,19.0,Perdio,518000.0
1,1,1,1,1.0,5.0,6.0,17000.0,20000.0,11.0,Perdio,518000.0
2,1,1,1,1.0,5.0,6.0,19000.0,20000.0,7.0,Perdio,518000.0
3,1,3,3,1.0,5.0,6.0,18000.0,20000.0,15.0,Perdio,518000.0
4,1,5,5,1.0,5.0,6.0,14000.0,20000.0,19.0,Perdio,518000.0
...,...,...,...,...,...,...,...,...,...,...,...
516,11,56,656,104.0,656.0,4.0,7000.0,10000.0,27.0,Perdio,26757250.0
517,11,56,656,104.0,656.0,4.0,2500.0,10000.0,48.0,Perdio,26757250.0
518,11,57,657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
519,11,57,657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
